In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
from tqdm import tqdm
import os
import data_utils
import model_utils
from attack_utils import get_CSMIA_case_by_case_results, CSMIA_attack, LOMIA_attack
from data_utils import oneHotCatVars, filter_random_data_by_conf_score
from vulnerability_score_utils import get_vulnerability_score, draw_hist_plot
from experiment_utils import MIAExperiment
from disparity_inference_utils import get_confidence_array, draw_confidence_array_scatter, get_indices_by_group_condition, get_corr_btn_sens_and_out_per_subgroup, get_slopes, get_angular_difference, calculate_stds, get_mutual_info_btn_sens_and_out_per_subgroup
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network._base import ACTIVATIONS
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import roc_curve, auc, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.decomposition import PCA
from sklearn.inspection import permutation_importance
from fairlearn.metrics import equalized_odds_difference, demographic_parity_difference
import matplotlib.pyplot as plt
import seaborn as sns
import tabulate
import pickle
# import utils
import copy

import matplotlib as mpl

# Setting the font family, size, and weight globally
mpl.rcParams['font.family'] = 'DejaVu Sans'
mpl.rcParams['font.size'] = 8
mpl.rcParams['font.weight'] = 'light'

In [4]:
experiment = MIAExperiment(sampling_condition_dict_list = 
    {
            'subgroup_col_name': 'PAT_STATUS',
            'subgroup_values': [1, 2, 3, 4, 6, 20, 50, 51, 62, 63],
            # 'subgroup_values': ['1', '2', '3', '4', '6', '20', '50', '51', '62', '63'],
            'n': 10000
    }, shortname = f"Corr_btn_sens_and_output_for_PAT_STATUS_ranging_from_0_to_-0.5", name='Texas100', sensitive_column='SEX_CODE'
)

 10%|█         | 1/10 [00:00<00:03,  2.25it/s]

before scaling: 2625 2188 2375
after scaling: 2188 2188 1.0
before scaling: 1979 1363 1.4519442406456347
after scaling: 1363 1363 1.0
before scaling: 2375 1990 2625
after scaling: 1990 1990 1.0
before scaling: 2199 1170 1.8794871794871795
after scaling: 1170 1170 1.0


 30%|███       | 3/10 [00:01<00:02,  2.56it/s]

before scaling: 2875 1347 2125
after scaling: 1347 1347 1.0
before scaling: 995 815 1.2208588957055215
after scaling: 815 815 1.0
before scaling: 2125 1800 2875
after scaling: 1800 1800 1.0
before scaling: 2435 1295 1.8803088803088803
after scaling: 1295 1295 1.0


 50%|█████     | 5/10 [00:01<00:01,  2.64it/s]

...

before scaling: 3250 1346 1750
after scaling: 1346 1346 1.0
before scaling: 724 693 1.0447330447330447
after scaling: 693 693 1.0
before scaling: 1750 1568 3250
after scaling: 1568 1568 1.0
before scaling: 2912 924 3.1515151515151514
after scaling: 924 924 1.0


 70%|███████   | 7/10 [00:02<00:01,  2.65it/s]

before scaling: 3375 1548 1625
after scaling: 1548 1548 1.0
before scaling: 745 640 1.1640625
after scaling: 640 640 1.0
before scaling: 3376 963 3.5057113187954307
after scaling: 963 963 1.0


100%|██████████| 10/10 [00:03<00:00,  2.62it/s]

before scaling: 3625 2288 1.5843531468531469
after scaling: 2288 2288 1.0
[5000, 5000, 2869, 2228, 5000, 5000, 1918, 2252, 5000, 5000, 3978, 2862, 1981, 1421, 1969, 1426, 5000, 5000, 5000, 3155]


In [6]:
save_model = True

print(f"Training classifier for experiment: {experiment}")
try:
    experiment.clf = model_utils.load_model(f'<PATH_TO_MODEL>/{experiment.ds.ds.filenameroot}_target_model.pkl')
    print(f"Loaded classifier for experiment from file: {experiment}")
except:
    # clf = model_utils.get_model(max_iter=500, hidden_layer_sizes=(256, 256))
    experiment.clf = model_utils.get_model(max_iter=500)
    experiment.clf.fit(experiment.X_train, experiment.y_tr_onehot)

    if save_model:
        model_utils.save_model(experiment.clf, f'<PATH_TO_MODEL>/{experiment.ds.ds.filenameroot}_target_model.pkl')

Training classifier for experiment: Texas100_subgroup_col_name_PAT_STATUS_subgroup_values_[1, 2, 3, 4, 6, 20, 50, 51, 62, 63]_n_10000


In [15]:

experiment.clf = model_utils.get_model(max_iter=500)
experiment.clf.fit(experiment.X_train, experiment.y_tr_onehot)

MLPClassifier(hidden_layer_sizes=(32, 16, 8), max_iter=500, random_state=42)

In [8]:
original_test_df = experiment.ds.ds.original_df[experiment.ds.ds.original_df['is_train']==1].copy().reset_index(drop=True).drop(['is_train'], axis=1)

sensitive_column = experiment.ds.ds.meta['sensitive_column']
y_column = experiment.ds.ds.meta['y_column']
subgroup_values = [1, 2, 3, 4, 6, 20, 50, 51, 62, 63]
p = [0] * 10
n = [original_test_df[original_test_df['PAT_STATUS']==i][[sensitive_column, y_column]].value_counts().to_numpy().min() * 4 for i in subgroup_values]

temp_indices = experiment.ds.ds.sample_data_matching_correlation(original_test_df, p=p, n=n, subgroup_col_name='PAT_STATUS', transformed_already=True, return_indices_only=True)

experiment.X_train_balanced_corr = experiment.X_train.loc[temp_indices].reset_index(drop=True)
experiment.y_tr_balanced_corr = experiment.y_tr[temp_indices]
experiment.y_tr_onehot_balanced_corr = experiment.y_tr_onehot[temp_indices]

[1, 2, 3, 4, 6, 20, 50, 51, 62, 63]
[10000, 4232, 9000, 3260, 8000, 4292, 1988, 1852, 6000, 3468]
{0: {(0, 1): 2500, (0, 0): 2500, (1, 1): 2500, (1, 0): 2500}, 1: {(0, 1): 1058, (0, 0): 1058, (1, 1): 1058, (1, 0): 1058}, 2: {(0, 1): 2250, (0, 0): 2250, (1, 1): 2250, (1, 0): 2250}, 3: {(0, 1): 815, (0, 0): 815, (1, 1): 815, (1, 0): 815}, 4: {(0, 1): 2000, (0, 0): 2000, (1, 1): 2000, (1, 0): 2000}, 5: {(0, 1): 1073, (0, 0): 1073, (1, 1): 1073, (1, 0): 1073}, 6: {(0, 1): 497, (0, 0): 497, (1, 1): 497, (1, 0): 497}, 7: {(0, 1): 463, (0, 0): 463, (1, 1): 463, (1, 0): 463}, 8: {(0, 1): 1500, (0, 0): 1500, (1, 1): 1500, (1, 0): 1500}, 9: {(0, 1): 867, (0, 0): 867, (1, 1): 867, (1, 0): 867}}


100%|██████████| 10/10 [00:00<00:00, 26.21it/s]


In [9]:
base_model = model_utils.get_model(max_iter=500)
experiment.clf_balanced_corr = copy.deepcopy(base_model)
experiment.clf_balanced_corr.fit(experiment.X_train_balanced_corr, experiment.y_tr_onehot_balanced_corr)

MLPClassifier(hidden_layer_sizes=(32, 16, 8), max_iter=500, random_state=42)

In [20]:
subgroup_oh_cols = [f'PAT_STATUS_{i}' for i in subgroup_values]
subgroup_vals_tr = np.array(subgroup_values)[experiment.X_test[subgroup_oh_cols].to_numpy().argmax(axis=1)]

In [21]:
subgroup_vals_tr

array([1, 1, 2, ..., 1, 1, 1])

In [42]:
def get_perf(experiment, clf, X_test, y_te, X_train, y_tr,  
    subgroup_col_name = 'PAT_STATUS', indices=None):
    y_tr_pred = np.argmax(clf.predict_proba(X_train), axis=1)

    subgroup_oh_cols = [f'{subgroup_col_name}_{i}' for i in subgroup_values]
    subgroup_vals_tr = np.array(subgroup_values)[X_train[subgroup_oh_cols].to_numpy().argmax(axis=1)]
    subgroup_vals_te = np.array(subgroup_values)[X_test[subgroup_oh_cols].to_numpy().argmax(axis=1)]
    # subgroup_vals_tr = X_train[f'{subgroup_col_name}_1'].to_numpy().ravel().astype(int)
    # subgroup_vals_te = X_test[f'{subgroup_col_name}_1'].to_numpy().ravel()

    sens_pred, case_indices = CSMIA_attack(clf, X_test, y_te, experiment.ds.ds.meta)
    sens_pred_LOMIA = LOMIA_attack(experiment, clf, X_test, y_te, experiment.ds.ds.meta, indices=indices)
    correct_indices = (sens_pred == X_test[[f'{experiment.ds.ds.meta["sensitive_column"]}_1']].to_numpy().ravel())
    correct_indices_LOMIA = (sens_pred_LOMIA == X_test[[f'{experiment.ds.ds.meta["sensitive_column"]}_1']].to_numpy().ravel())

    # male_indices = np.where(subgroup_vals_tr==0)[0]
    # female_indices = np.where(subgroup_vals_tr==1)[0]
    
    return [correct_indices[subgroup_vals_te==i].mean() for i in subgroup_values]
    perf_dict = {
        'ASRD_CSMIA': round(100 * np.ptp([correct_indices[subgroup_vals_te==i].mean() for i in subgroup_values]), 2),
        'ASRD_LOMIA': round(100 * np.ptp([correct_indices_LOMIA[subgroup_vals_te==i].mean() for i in subgroup_values]), 2),
        'EOD': round(equalized_odds_difference(y_tr.ravel(), y_tr_pred, sensitive_features=subgroup_vals_tr), 4),
        'DPD': round(demographic_parity_difference(y_tr.ravel(), y_tr_pred, sensitive_features=subgroup_vals_tr), 4),
        'MA': 100 * accuracy_score(y_tr.ravel()[:], y_tr_pred[:])
        # 'MA_Male': 100 * accuracy_score(y_tr.ravel()[male_indices], y_tr_pred[male_indices]),
        # 'MA_Female': 100 * accuracy_score(y_tr.ravel()[female_indices], y_tr_pred[female_indices])
    }

    return perf_dict


In [43]:
asrs = get_perf(experiment, experiment.clf, experiment.X_train, experiment.y_tr, experiment.X_test, experiment.y_te)

In [44]:
asrs_after = get_perf(experiment, experiment.clf_balanced_corr, experiment.X_train_balanced_corr, experiment.y_tr_balanced_corr, experiment.X_test, experiment.y_te, indices=temp_indices)

In [46]:
100 * (np.max(asrs))

72.59350091968118

In [47]:
100 * (np.max(asrs_after))

59.07127429805615

In [18]:
np.ptp(asrs)

0.22838256746900076

In [88]:
model_arch = hidden_layer_size_tuples[1]
res_dict = {
    'w/o BCorr': get_perf(experiment, experiment.clfs[model_arch], experiment.X_test, experiment.y_te, experiment.X_train, experiment.y_tr),
    'w Bcorr': get_perf(experiment, experiment.clfs_balanced_corr[model_arch], experiment.X_test_balanced_corr, experiment.y_te_balanced_corr, experiment.X_train, experiment.y_tr, indices=temp_indices)
}
res_dict_df = pd.DataFrame.from_dict(res_dict, orient='index')

In [89]:
res_dict_df

,ASRD_CSMIA,ASRD_LOMIA,EOD,DPD,MA_Male,MA_Female
w/o BCorr,12.52,7.48,0.0674,0.0178,73.208,78.16
w Bcorr,2.06,2.28,0.0416,0.0015,73.888,77.90


In [90]:
print(res_dict_df.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
 & ASRD_CSMIA & ASRD_LOMIA & EOD & DPD & MA_Male & MA_Female \\
\midrule
w/o BCorr & 12.520000 & 7.480000 & 0.067400 & 0.017800 & 73.208000 & 78.160000 \\
w Bcorr & 2.060000 & 2.280000 & 0.041600 & 0.001500 & 73.888000 & 77.900000 \\
\bottomrule
\end{tabular}



In [64]:
res_dict

{'w/o BCorr': {'MA_Female': 78.16,
  'MA_Male': 73.208,
  'EOD': 0.06735999999999998,
  'DPD': 0.017839999999999967,
  'ASRD': 0.12524000000000002},
 'w Bcorr': {'MA_Female': 77.9,
  'MA_Male': 73.888,
  'EOD': 0.04160000000000008,
  'DPD': 0.0014800000000000368,
  'ASRD': 0.020580823232929313}}

In [61]:
for model_arch in experiment.clfs:
    print(get_perf(experiment, experiment.clfs[model_arch], experiment.X_test, experiment.y_te, experiment.X_train, experiment.y_tr))

{'MA_Female': 77.164, 'MA_Male': 72.712, 'EOD': 0.057999999999999996, 'DPD': 0.013480000000000047, 'ASRD': 0.08372000000000002}
{'MA_Female': 78.16, 'MA_Male': 73.208, 'EOD': 0.06735999999999998, 'DPD': 0.017839999999999967, 'ASRD': 0.12524000000000002}
{'MA_Female': 78.99199999999999, 'MA_Male': 73.628, 'EOD': 0.06672, 'DPD': 0.013080000000000092, 'ASRD': 0.14991999999999994}
{'MA_Female': 75.932, 'MA_Male': 72.348, 'EOD': 0.03735999999999995, 'DPD': 0.0015199999999999658, 'ASRD': 0.05384}


In [62]:
for model_arch in experiment.clfs:
    print(get_perf(experiment, experiment.clfs_balanced_corr[model_arch], experiment.X_test_balanced_corr, experiment.y_te_balanced_corr, experiment.X_train, experiment.y_tr))

{'MA_Female': 76.728, 'MA_Male': 72.272, 'EOD': 0.04927999999999999, 'DPD': 0.0047200000000000575, 'ASRD': 0.01446057842313686}
{'MA_Female': 77.9, 'MA_Male': 73.888, 'EOD': 0.04160000000000008, 'DPD': 0.0014800000000000368, 'ASRD': 0.020580823232929313}
{'MA_Female': 78.532, 'MA_Male': 75.312, 'EOD': 0.049360000000000015, 'DPD': 0.017159999999999953, 'ASRD': 0.004560182407296276}
{'MA_Female': 75.62, 'MA_Male': 72.38799999999999, 'EOD': 0.03527999999999998, 'DPD': 0.0029599999999999627, 'ASRD': 0.0006000240009600422}


In [12]:
get_corr_btn_sens_and_out_per_subgroup(experiment, experiment.X_test_balanced_corr, experiment.y_te_balanced_corr, {'SEX': 0})

0.10008400408079707